# Proeycto de IA - ChatBox

Departamento de Ciencias de la Computación, Universidad de Chile.

CC6409: Taller de Desarrollo de Poryectod e IA - Otoño 2023

**Integrantes:**
- Garrido Martín
- Gómez Nahuel
- Santelices Gustavo


## Import

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np
import nlpaug.augmenter.word as naw # To synonym, see  to more info https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
from mtranslate import translate
import pickle

c:\Users\nahue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## data

In [2]:
# handmade
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

hand_tavo_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_tavo.csv", sep = ';')
hand_tavo_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_tavo.csv", sep = ';')
hand_tavo_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_tavo.csv", sep = ';')
hand_tavo_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_tavo.csv", sep = ';')

hand_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_martin.csv", sep = ';', encoding = 'latin-1')
hand_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_martin.csv", sep = ';')
hand_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_martin.csv", sep = ';')
hand_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_martin.csv", sep = ';')

In [3]:
df_ = pd.concat([hand_nawel_0, hand_nawel_1, hand_nawel_2, hand_nawel_3,
                 hand_tavo_0, hand_tavo_1, hand_tavo_2, hand_tavo_3,
                 hand_martin_0, hand_martin_1, hand_martin_2, hand_martin_3])

df_.count()

class    345
text     345
dtype: int64

In [4]:
df_['class'].value_counts()

class
0    101
1     89
2     87
3     68
Name: count, dtype: int64

## Data Augmentation (in a first step, to all parameters)

### ChatGPT sentences

In [5]:
# Nahuel
gpt_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_nawel.csv", sep = ';')
gpt_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_nawel.csv", sep = ';')
gpt_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_nawel.csv", sep = ';')
gpt_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_nawel.csv", sep = ';')

# Martin
gpt_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_martin.csv", sep = ';')
gpt_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_martin.csv", sep = ';')
gpt_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_martin.csv", sep = ';')
gpt_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_martin.csv", sep = ';')

# Gustavo
#gpt_tavo_0 = pd.read_csv("", sep = ';')
#gpt_tavo_1 = pd.read_csv("", sep = ';')
#gpt_tavo_2 = pd.read_csv("", sep = ';')
#gpt_tavo_3 = pd.read_csv("", sep = ';')

In [6]:
# Concat the chatgpt sentences
df_ = pd.concat([df_, 
                 gpt_nawel_0, gpt_nawel_1, gpt_nawel_2, gpt_nawel_3,
                 gpt_martin_0, gpt_martin_1, gpt_martin_2, gpt_martin_3])

In [7]:
df_['class'].value_counts()

class
0    185
1    178
2    162
3    141
Name: count, dtype: int64

### Translate augmentation (functions)

In [10]:
# Extra : Translate Ideas
def sp_to_en(data):
    augmented_data = []
    for sentence in data:
        # Translate Spanish sentence to English
        translated_sentence = translate(sentence, "en")
        augmented_data.append(translated_sentence)

    return augmented_data

def en_to_sp(data):
    augmented_data = []
    for sentence in data:
        # Translate English sentence to Spanish
        translated_sentence = translate(sentence, "es")

        augmented_data.append(translated_sentence)

    return augmented_data

In [11]:
# Import model
aug_fb = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

In [12]:
def Translate_aug(df_):
    # Getting the values with two or more words.
    df_english_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 2)]

    # Traslate to English
    l_english_aug = sp_to_en(df_english_aug.text.tolist())

    # Find Synonyms
    english_syn_aug = aug_fb.augment(l_english_aug)

    # Translate to spanish
    l_english_aug = en_to_sp(df_english_aug.text.tolist())

    # Append text with label
    df_english = pd.DataFrame()
    df_english['class'] = df_english_aug['class']
    df_english['text'] = l_english_aug

    return df_english

### Data aug in spanish (functions)

In [13]:
# Wordnet - spa
aug_wn = naw.SynonymAug(aug_src='wordnet', lang='spa')

In [14]:
def spanish_aug(df_):
    # Getting the values with ten or more words.
    df_spanish_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 10)]
    l_augmented = aug_wn.augment(df_spanish_aug.text.tolist())

    # Append text with label
    df_spanish = pd.DataFrame()
    df_spanish['class'] = df_spanish_aug['class']
    df_spanish['text'] = l_augmented

    return df_spanish

## Custom Features

In [15]:
class PreProccesingTransformer(BaseEstimator, TransformerMixin):
    def preprocess(self,sentence):
      # Deleting all except: exclamation/question signs and accents
      new_word = re.sub(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ¡!¿?\s]", '', sentence)
      # Deleting double blank spaces
      new_sentence = new_word.replace('  ',' ').replace('\n','').strip()
      return new_sentence

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.preprocess(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

## Pipeline

### Run Pipeline

In [16]:
# Defining threshold
thresholds = {
    0: 0.90,  # class 0
    1: 0.80,  # class 1
    2: 0.80,   # class 2
    3: 0.80   #class 3
}


def custom_predict(model, X, class_thresholds):
    # Get probabilities
    probabilities = model.predict_proba(X)

    # Aplied threshold
    modified_predictions = np.argmax(probabilities, axis=1)
    for class_label, threshold in class_thresholds.items():
        modified_predictions[probabilities[:, class_label] > threshold] = class_label

    return modified_predictions

In [17]:
def run(dataset, pipeline):

    X_train, X_test, y_train, y_test = train_test_split(
        dataset['text'],
        dataset['class'],
        shuffle=True,
        test_size=0.2,
        random_state=42,
        stratify=dataset['class']
    )

    print(f"# Len Training Data: {len(X_train)}")
    print(f"# Len Testing Data: {len(X_test)}")

    print('Aplying data aug')

    df_to_aug = pd.DataFrame({'text': X_train, 'class': y_train})

    english_df = Translate_aug(df_to_aug)
    spanish_df = spanish_aug(df_to_aug)

    X_train = pd.concat([X_train, english_df['text'], spanish_df['text']])
    y_train = pd.concat([y_train, english_df['class'], spanish_df['class']])

    print(f"# NEW Len Training Data: {len(X_train)}")
    print(f"# NEW Len Testing Data: {len(X_test)}")

    pipeline.fit(X_train, y_train)


    # Classes propbailities
    probabilities = pipeline.predict_proba(X_test)


    # Predicted labels
    predicted_labels = pipeline.predict(X_test)
    #predicted_labels = custom_predict(pipeline, X_test, thresholds)


    print(classification_report(y_test, predicted_labels))


    return pipeline, X_train, X_test, y_train, y_test, predicted_labels, probabilities


## Defining Pipelines

In [18]:
# simple Pipeline only BOW

def get_experiment_0_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [19]:
# simple Pipeline only Preprocessing + BOW

def get_experiment_1_pipeline():

    return Pipeline(
        [
            ("preprocessing", PreProccesingTransformer()),
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer())
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

## Implementación del modelo

In [20]:
# Simple Pipeline Test
pipeline0 = get_experiment_0_pipeline()

prob = run(df_, pipeline0)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        37
           1       0.84      0.86      0.85        36
           2       0.79      0.94      0.86        33
           3       0.93      0.89      0.91        28

    accuracy                           0.87       134
   macro avg       0.87      0.87      0.87       134
weighted avg       0.87      0.87      0.87       134



In [27]:
print(pipeline0.predict(['Cancela mi hora']))

[1]


In [21]:
# Save the model
with open('model0.bin', 'wb') as fil:
    pickle.dump(pipeline0, fil)

In [28]:
import pickle
from sup_file import PreProccesingTransformer

with open(r"C:\Users\nahue\OneDrive\Escritorio\Universidad\9no_Semestre\CC6409_Proyecto de IA\ProyectoDeIA\NLP_model\model0.bin", 'rb') as archivo:
    modelo = pickle.load(archivo)

print(modelo.predict(['Cancela mi hora']))


[1]


In [22]:
# Simple Pipeline Test (+ preprocessing)
pipeline1 = get_experiment_1_pipeline()

_ = run(df_, pipeline1)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug


KeyboardInterrupt: 

In [ ]:
# Save the model
with open('model1.bin', 'wb') as fil:
    pickle.dump(pipeline1, fil)

In [ ]:
## Respuesta al usuario

In [ ]:
map_label= {
    0: 'Asistir a la',
    1: 'Cancelar la',
    2: 'Reagendar la',
    3: 'Pedir una'
}

options = {
    0: 'Su reserva a quedado confirmada para el DD de MM',
    1: 'Su reserva con fecha DD de MM a quedado cancelada',
    2: 'Las fechas para reagendar son las siguientes',
    3: 'Las fechas próximas son las siguientes'
}

def manage_mesages(target):
    # Make a prediction
    label = pipeline1.predict([target])[0]
    print(f"usted quiere {map_label[label]} hora")
    print(options[label])

In [ ]:
manage_mesages('Puedo cancelar la hora?')

usted quiere Cancelar la hora
Su reserva con fecha DD de MM a quedado cancelada
